In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
import json
import urllib.request
#import polygon as settings 
from datetime import date, datetime
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from polygon import RESTClient
from local_settings import polygon as settings
import hvplot.pandas
import seaborn as sns
%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
# Load API KEY from env file
polygon_api_key = os.getenv("api_key")
print(polygon_api_key)

In [ ]:
markets = ['crypto', 'stocks', 'fx']

class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)
        


In [ ]:
client = MyRESTClient(settings['api_key'])

In [ ]:
help(client.reference_tickers_v3)

In [ ]:
class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.</em>
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

In [ ]:
client = MyRESTClient(settings['api_key'])
df = client.get_tickers(market='crypto')
df

In [ ]:
# Default example
df2 = df.where(df.ticker == 'X:1INCHUSD')
df2

In [ ]:
help(client.stocks_equities_aggregates)

In [ ]:
class MyRESTClient(RESTClient):
    def __init__(self, auth_key: str=settings['api_key'], timeout:int=5):
        super().__init__(auth_key)
        retry_strategy = Retry(total=10,
                               backoff_factor=10,
                               status_forcelist=[429, 500, 502, 503, 504])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self._session.mount('https://', adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                # Only use USD pairings.</em>
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

    def get_bars(self, market:str=None, ticker:str=None, multiplier:int=1,
                 timespan:str='minute', from_:date=None, to:date=None) -> pd.DataFrame:

        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        if ticker is None:
            raise Exception('Ticker must not be None.')

        from_ = from_ if from_ else date(2000,1,1)
        to = to if to else date.today()

        if market == 'crypto':
            resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          from_.strftime('%Y-%m-%d'), to.strftime('%Y-%m-%d'),
                                          limit=50000)
            df = pd.DataFrame(resp.results)
            last_minute = 0
            while resp.results[-1]['t'] > last_minute:
                last_minute = resp.results[-1]['t'] 
                # Last minute in response
                last_minute_date = datetime.fromtimestamp(last_minute/1000).strftime('%Y-%m-%d')
                resp = self.crypto_aggregates(ticker, multiplier, timespan,
                                          last_minute_date, to.strftime('%Y-%m-%d'),
                                          limit=50000)
                new_bars = pd.DataFrame(resp.results)
                df = df.append(new_bars[new_bars['t'] > last_minute])

            df['date'] = pd.to_datetime(df['t'], unit='ms')
            df = df.rename(columns={'o':'open',
                                    'h':'high',
                                    'l':'low',
                                    'c':'close',
                                    'v':'volume',
                                    'vw':'vwap',
                                    'n':'transactions'})
            df = df[['date','open','high','low','close','volume']]

            return df
        return None

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
BTC_df = client.get_bars(market='crypto', ticker='X:BTCUSD', from_=start)


In [ ]:
BTC_df.set_index('date')

In [ ]:
BTC_df.hvplot(x='date', y=['high', 'close', 'volume'],
                value_label='Price', subplots=True, width=600, height=400)

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
ETH_df = client.get_bars(market='crypto', ticker='X:ETHUSD', from_=start)


In [ ]:
ETH_df.set_index('date') 

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
SOL_df = client.get_bars(market='crypto', ticker='X:SOLUSD', from_=start)
SOL_df

In [ ]:
SOL_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
MATIC_df = client.get_bars(market='crypto', ticker='X:MATICUSD', from_=start)
MATIC_df

In [ ]:
MATIC_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
XRP_df = client.get_bars(market='crypto', ticker='X:XRPUSD', from_=start)
XRP_df

In [ ]:
XRP_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
XLM_df = client.get_bars(market='crypto', ticker='X:XLMUSD', from_=start)
XLM_df

In [ ]:
XLM_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
DOGE_df = client.get_bars(market='crypto', ticker='X:DOGEUSD', from_=start)
DOGE_df

In [ ]:
DOGE_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
SHIB_df = client.get_bars(market='crypto', ticker='X:SHIBUSD', from_=start)
SHIB_df

In [ ]:
SHIB_df.set_index('date')

In [ ]:
start = datetime(2017,1,1)
client = MyRESTClient(settings['api_key'])
MANA_df = client.get_bars(market='crypto', ticker='X:MANAUSD', from_=start)


In [ ]:
MANA_df.set_index('date')

In [ ]:
df_meme_crypto_1 = pd.merge(DOGE_df,MANA_df,on='date',how='inner')
df_meme_crypto_1

In [ ]:
df_meme_crypto2 = pd.merge(df_meme_crypto_1,SHIB_df,on='date',how='inner')
df_meme_crypto2

In [ ]:
df_meme_crypto2.columns =['Date','DOGE_open', 'DOGE_high', 'DOGE_low', 'DOGE_close', 'DOGE_Vol', 'MANA_open', 'MANA_high', 'MANA_low', 'MANA_close', 'MANA_Vol', 'SHIB_open', 'SHIB_high', 'SHIB_low', 'SHIB_close', 'SHIB_Vol']

In [ ]:
df_meme_crypto2.set_index('Date')

In [ ]:
df_meme = df_meme_crypto2[['Date','DOGE_close','MANA_close','SHIB_close']]

In [ ]:
df_meme.head(30)

In [ ]:
df_meme.hvplot(x='Date', y=['DOGE_close', 'MANA_close', 'SHIB_close'],
                value_label='Price', subplots=True, width=600, height=400)

In [ ]:
df_meme_pct_change = df_meme.set_index('Date').pct_change()

In [ ]:
df_meme_pct_change.hvplot(figsize = (20,10))

In [ ]:
# Calculate the correlation of closing prices between portfolios
price_correlation = df_meme_pct_change.corr()

# Use the `heatmap` function from the seaborn library to visualize correlations
sns.heatmap(price_correlation, vmin=0, vmax=1)

# Refer to markdown for analysis

price_correlation

In [ ]:
#Calculate the weighted returns for your portfolio, assuming equal number of shares per stock.
# Set weights for corresponding of stocks: GOOG, AAL, NVDA, TSLA, ICNL at 20%
weights = [0.34, 0.33, 0.33]

# Use the `dot` function to sum the product each weight and the corresponding daily return.
memeportfolio_returns = df_meme_pct_change.dot(weights)

# Diplay sample values
memeportfolio_returns.plot(figsize = (20,10))

# Use the `cumprod` function to calculate cumulative returns
memecumulative_returns = (1 + memeportfolio_returns).cumprod()

# Display sample values
memecumulative_returns.head()

# Set initial investment
initial_investment = 100000

# Compute cumulative returns
memecumulative_profit = initial_investment * memecumulative_returns

# Plot the returns of the investment
memecumulative_profit.hvplot(figsize=(20,10))

In [ ]:
memecumulative_profit

In [ ]:
df_coin_crypto_1 = pd.merge(BTC_df,XRP_df,on='date',how='inner')
df_coin_crypto_1

In [ ]:
df_coin_crypto2 = pd.merge(df_coin_crypto_1,XLM_df,on='date',how='inner')
df_coin_crypto2

In [ ]:
df_coin_crypto2.columns =['Date','BTC_open', 'BTC_high', 'BTC_low', 'BTC_close', 'BTC_Vol',
                          'XRP_open', 'XRP_high', 'XRP_low', 'XRP_close', 'XRP_Vol', 
                          'XLM_open', 'XLM_high', 'XLM_low', 'XLM_close', 'XLM_Vol']

df_coin_crypto2

In [ ]:
df_coin = df_coin_crypto2[['Date','BTC_close','XRP_close','XLM_close']]
df_coin

In [ ]:
df_coin.hvplot(x='Date', y=['BTC_close', 'XRP_close', 'XLM_close'],
                value_label='Price', subplots=True, width=600, height=400)

In [ ]:
df_coin_pct_change = df_coin.set_index('Date').pct_change()
df_coin_pct_change

In [ ]:
df_coin_pct_change.hvplot(figsize = (20,10))

In [ ]:
# Calculate the correlation of closing prices between portfolios
price_correlation_coin = df_coin_pct_change.corr()

# Use the `heatmap` function from the seaborn library to visualize correlations
sns.heatmap(price_correlation_coin, vmin=0, vmax=1)

# Refer to markdown for analysis

price_correlation_coin

In [ ]:
#Calculate the weighted returns for your portfolio, assuming equal number of shares per stock.
# Set weights for corresponding of stocks: GOOG, AAL, NVDA, TSLA, ICNL at 20%
weights = [0.34, 0.33, 0.33]

# Use the `dot` function to sum the product each weight and the corresponding daily return.
coin_portfolio_returns = df_coin_pct_change.dot(weights)

# Diplay sample values
coin_portfolio_returns.plot(figsize = (20,10))

# Use the `cumprod` function to calculate cumulative returns
coin_cumulative_returns = (1 + coin_portfolio_returns).cumprod()

# Display sample values
coin_cumulative_returns.head()

# Set initial investment
initial_investment = 100000

# Compute cumulative returns
coin_cumulative_profit = initial_investment * coin_cumulative_returns

# Plot the returns of the investment
coin_cumulative_profit.hvplot(figsize=(20,10))

In [ ]:
coin_cumulative_profit

In [ ]:
df_chain_crypto_1 = pd.merge(ETH_df,SOL_df,on='date',how='inner')
df_chain_crypto_1

In [ ]:
df_chain_crypto2 = pd.merge(df_chain_crypto_1,MATIC_df,on='date',how='inner')
df_chain_crypto2

In [ ]:
df_chain_crypto2.columns =['Date','ETH_open', 'ETH_high', 'ETH_low', 'ETH_close', 'ETH_Vol',
                          'SOL_open', 'SOL_high', 'SOL_low', 'SOL_close', 'SOL_Vol', 
                          'MATIC_open', 'MATIC_high', 'MATIC_low', 'MATIC_close', 'MATIC_Vol']

df_chain_crypto2

In [ ]:
df_chain = df_chain_crypto2[['Date','ETH_close','SOL_close','MATIC_close']]
df_chain

In [ ]:
df_chain_pct_change = df_chain.set_index('Date').pct_change()
df_chain_pct_change

In [ ]:
df_chain_pct_change.hvplot(figsize = (20,10))

In [ ]:
# Calculate the correlation of closing prices between portfolios
price_correlation_chain = df_chain_pct_change.corr()

# Use the `heatmap` function from the seaborn library to visualize correlations
sns.heatmap(price_correlation_chain, vmin=0, vmax=1)

# Refer to markdown for analysis

price_correlation_chain

In [ ]:
#Calculate the weighted returns for your portfolio, assuming equal number of shares per stock.
# Set weights for corresponding of stocks: GOOG, AAL, NVDA, TSLA, ICNL at 20%
weights = [0.34, 0.33, 0.33]

# Use the `dot` function to sum the product each weight and the corresponding daily return.
chain_portfolio_returns = df_chain_pct_change.dot(weights)

# Diplay sample values
chain_portfolio_returns.plot(figsize = (20,10))

# Use the `cumprod` function to calculate cumulative returns
chain_cumulative_returns = (1 + chain_portfolio_returns).cumprod()

# Display sample values
chain_cumulative_returns.head()

# Set initial investment
initial_investment = 100000

# Compute cumulative returns
chain_cumulative_profit = initial_investment * chain_cumulative_returns


# Plot the returns of the investment
chain_cumulative_profit.hvplot(figsize=(20,10))



In [ ]:
chain_cumulative_profit